# Importing Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Library Imports

In [ ]:
import pandas as pd
import numpy as np
import itertools
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.ar_model import AutoReg
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from statsmodels.tsa.stattools import pacf
# import pandoc

# Reading Data

## Reading train and test data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Time Series Analysis/Data/train.csv")
df["date"] = pd.to_datetime(df['date'], format='%Y-%m-%d') # formatting the date

In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/Time Series Analysis/Data/test.csv")
df_test["date"] = pd.to_datetime(df_test['date'], format='%Y-%m-%d') # formatting the date

In [ ]:
df_test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [ ]:
df_final = pd.DataFrame(columns=['id','date','store_nbr','family','onpromotion','pred'])

## Reading store data

In [ ]:
df_stores = pd.read_csv('/content/drive/MyDrive/Time Series Analysis/Data/stores.csv')

In [ ]:
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


# Creating Combination of store number, cluster & product family

In [ ]:
#Fetching list of unique products and store numbers
family_list = df['family'].unique().tolist()
store_list = df['store_nbr'].unique().tolist()

In [ ]:
comb_store_family = list(itertools.product(store_list, family_list))
len(comb_store_family)

1782

# Merging of store numbers with their respective cluster

In [ ]:
df_merge = pd.merge(df, df_stores[['store_nbr','cluster']], how='left', on='store_nbr')

In [ ]:
df_merge.head()

,id,date,store_nbr,family,sales,onpromotion,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,13
1,1,2013-01-01,1,BABY CARE,0.0,0,13
2,2,2013-01-01,1,BEAUTY,0.0,0,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,13
4,4,2013-01-01,1,BOOKS,0.0,0,13


# Checking lags using PACF

In [ ]:
family_list

['AUTOMOTIVE',
 'BABY CARE',
 'BEAUTY',
 'BEVERAGES',
 'BOOKS',
 'BREAD/BAKERY',
 'CELEBRATION',
 'CLEANING',
 'DAIRY',
 'DELI',
 'EGGS',
 'FROZEN FOODS',
 'GROCERY I',
 'GROCERY II',
 'HARDWARE',
 'HOME AND KITCHEN I',
 'HOME AND KITCHEN II',
 'HOME APPLIANCES',
 'HOME CARE',
 'LADIESWEAR',
 'LAWN AND GARDEN',
 'LINGERIE',
 'LIQUOR,WINE,BEER',
 'MAGAZINES',
 'MEATS',
 'PERSONAL CARE',
 'PET SUPPLIES',
 'PLAYERS AND ELECTRONICS',
 'POULTRY',
 'PREPARED FOODS',
 'PRODUCE',
 'SCHOOL AND OFFICE SUPPLIES',
 'SEAFOOD']

In [ ]:
cluster_mapping = df_merge[['store_nbr',"cluster","family"]].groupby(['store_nbr',"cluster","family"]).count().reset_index()

In [ ]:
cluster_mapping.head()

,store_nbr,cluster,family
0,1,13,AUTOMOTIVE
1,1,13,BABY CARE
2,1,13,BEAUTY
3,1,13,BEVERAGES
4,1,13,BOOKS


In [ ]:
#for reference average lags per cluster
cluster_lag_map = {1:14,2:18,3:8,4:13,5:14,6:7,7:10,8:10,9:15,10:9,11:7,12:14,13:21,14:8,15:9,16:8,17:7}

1.) Remove the negative sign → convert positive the output of pacf 
2.) Drop all the values less than 0.05 → remember the index 
3.) Compute the diff between the pacf value and 0.05
4.) Compute the cumulative sum 
5.) Pick a threshold value - % change in cumulative sum < picked value then stop 
6.) Pass this index as the “p” value.. Similar approach for ACF  


In [ ]:

list_df = cluster_mapping.values.tolist()
list_cluster = []
for i in list_df:
  df_temp = df_merge[(df_merge['store_nbr']==i[0])&(df_merge['family']==i[2])]
  if df_temp["sales"].sum() > 0:
    pacf_test = pacf(df_temp['sales'].values,method='ywadjusted',nlags=20)
    df_pacf = pd.DataFrame(pacf_test, columns=['pacf_value'])
    df_pacf["abs_pacf"]= df_pacf["pacf_value"].abs()
    df_pacf.drop(df_pacf[df_pacf.abs_pacf < 0.05].index, inplace=True)
    df_pacf["difference"] = df_pacf.abs_pacf - 0.05
    df_pacf['cum_sum'] = df_pacf['difference'].cumsum()
    df_pacf['div'] = df_pacf['cum_sum'].diff()/df_pacf['cum_sum']
    df_pacf = df_pacf.loc[1:,:]
    list_index = df_pacf[(df_pacf["div"] >= df_pacf["div"].mean()) & (df_pacf.index > 7)].index.values.tolist()
    if len(list_index) > 0:
      i.append(list_index[0])
    else:
      i.append(7)
  else:
    i.append(1)
  list_cluster.append(i)

In [ ]:
list_cluster 

In [ ]:
#from statsmodels.tsa.stattools import pacf
#pacf_test = pacf(df_temp['sales'].values,method='ywadjusted',nlags=60,alpha=None)
#pacf_test

In [ ]:
#from statsmodels.tsa.stattools import kpss
#kpss_scores = kpss(df_temp['sales'].values,regression='c', nlags=1, store=False)
#kpss_scores

In [ ]:
df_merge.shape

(3000888, 7)

# Modeling AR

In [ ]:
for i in list_cluster:
  print("NEW ITERATION")
  print(i)
  df_temp = df_merge[(df_merge['store_nbr']==i[0])&(df_merge['family']==i[2])].reset_index(drop = True)
  df_temp["sales"] = np.log(df_temp["sales"]) #logarithmic normalization
  df_temp = df_temp.fillna(0) #filling null values
  df_temp = df_temp.replace([np.inf, -np.inf], 0) #filling values when log(0) takes place which cannot be computed
  df_test_temp =  df_test[(df_test['store_nbr']==i[0])&(df_test['family']==i[2])].reset_index(drop = True)
  if len(df_test_temp)>0:
    ar_model = AutoReg(df_temp["sales"], lags=i[3]).fit() #fitting the model
    df_test_temp["sales"] = np.exp(ar_model.predict(start=len(df_temp), end=(len(df_temp)+len(df_test_temp)-1), dynamic=False).reset_index(drop=True).values)
    df_final = pd.concat([df_final, df_test_temp])

In [ ]:
df_final.head()

,id,date,store_nbr,family,onpromotion,pred,sales
0,3000888,2017-08-16,1,AUTOMOTIVE,0,NaN,3.581187
1,3002670,2017-08-17,1,AUTOMOTIVE,0,NaN,3.391863
2,3004452,2017-08-18,1,AUTOMOTIVE,0,NaN,2.596307
3,3006234,2017-08-19,1,AUTOMOTIVE,0,NaN,3.397346
4,3008016,2017-08-20,1,AUTOMOTIVE,0,NaN,2.502870


## Joining test set with predict to submission file based on id column

In [ ]:
df_sample = pd.read_csv("/content/drive/MyDrive/ML Project /Time Series Analysis/Data/sample_submission.csv")
# df_sample["sales"] = df_final["pred"].values
df_sample_merge = pd.merge(df_sample["id"], df_final[['id','sales']], how='left', on='id')
df_sample_merge.head()

,id,sales
0,3000888,3.581187
1,3000889,1.000000
2,3000890,2.761529
3,3000891,1722.760048
4,3000892,1.011576


## Saving the submission file to a drive

In [ ]:
df_sample_merge.to_csv("/content/drive/MyDrive/ML Project /Time Series Analysis/Submission files/AR_model_v2_sub.csv",index=False)